In [1]:
import numpy as np

In [2]:
captions = np.load('./DATASETS/spanish_train.npy')

In [3]:
captions

array([{'image_id': 318556, 'caption': 'Un cuarto de baño muy limpio y bien decorado vacío'},
       {'image_id': 116100, 'caption': 'Una vista panorámica de una cocina y todos sus aparatos.'},
       {'image_id': 318556, 'caption': 'Un cuarto de baño azul y blanco con azulejos de la pared temáticos de la mariposa.'},
       ...,
       {'image_id': 180285, 'caption': 'Dos mujeres se sientan y posan con animales de peluche.'},
       {'image_id': 133071, 'caption': 'Plato blanco con una gran cantidad de guacamole y una cucharada grande extra de crema agria sobre la carne'},
       {'image_id': 133071, 'caption': 'Un plato tiene un embargo de cuña de limón.'}],
      dtype=object)

In [4]:
import utils
import params
import numpy as np

from tqdm import tqdm

In [5]:
import io

def load_vectors(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    #n, d = map(int, fin.readline().split())
    data = {}
    for line in fin:
        tokens = line.rstrip().split(' ')
        data[tokens[0]] = [float(x) for x in tokens[1:]]
    return data

In [6]:
fname = 'MODEL/cc.es.300.vec'
vectors = load_vectors(fname)

In [7]:
def reCaption(captions):
    from collections import defaultdict
    from sklearn.feature_extraction.text import CountVectorizer
    from nltk.corpus import stopwords
    recaptions = defaultdict(list)
    stop_words = set (stopwords.words( 'spanish' ))
    unigram_vectorizer = CountVectorizer()
    analyze = unigram_vectorizer.build_analyzer()

    for caption in captions:
        name = caption.get('image_id')
        des = caption.get('caption').lower()

        #remove stopwords
        des_stemmed = []
        for word in analyze(des):
            if word not in stop_words:
                des_stemmed.append(word)
        des_stemmed = ' '.join(des_stemmed)
        
        recaptions[name].append(des_stemmed)
    return recaptions

In [8]:
recaptions = reCaption(captions)

In [9]:
def applyword2vec(captions, vectors, path):
    from sklearn.feature_extraction.text import CountVectorizer
    unigram_vectorizer = CountVectorizer()
    analyze = unigram_vectorizer.build_analyzer()
    
    i = 0
    for caption in tqdm(captions.keys()):

        sentences = captions[caption]
        caption_descriptors = []
        for sentence in sentences:
            word_descriptor = []
            for word in analyze(sentence):
                if vectors.get(word) is not None:
                    word_descriptor.append( np.asarray(vectors[word] ))
                else:
                    word_descriptor.append( np.zeros(300))

            sentence_descriptor = np.mean(word_descriptor, axis=0)
            caption_descriptors.append(sentence_descriptor)
            name = path+str(caption)+'.npy'
        np.save(name,np.asarray(caption_descriptors))    

In [10]:
applyword2vec(recaptions, vectors, 'C:\\Users\\bldr1\\Desktop\\TESIS\\DESCRIPTORS\\WORD2VECT_SPANISH\\TRAIN\\')

100%|██████████| 82783/82783 [09:44<00:00, 141.67it/s]
